### Import Dependencies (Pandas)

In [2]:
import pandas as pd
import time

### Load Data

In [3]:
begin = time.time()
df = pd.read_csv("E:/workspace/bank-data-challenge/data/Consumer_Complaints.csv",index_col=False)
print(f"Elapsed: {round(time.time()-begin, 2)} sec")

Elapsed: 7.42 sec


### Profile Data

In [4]:
df.head(10)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,08/09/2015,Credit reporting,NaN,Incorrect information on credit report,Information is not mine,NaN,Company chooses not to provide a public response,Experian Information Solutions Inc.,NJ,08872,NaN,Consent not provided,Web,08/09/2015,Closed with non-monetary relief,Yes,No,1509954
1,01/29/2019,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Was not notified of investigation status or re...,NaN,NaN,"EQUIFAX, INC.",NY,10801,NaN,Consent not provided,Web,01/30/2019,Closed with non-monetary relief,Yes,NaN,3136759
2,10/13/2019,Debt collection,I do not know,Attempts to collect debt not owed,Debt was result of identity theft,NaN,NaN,SANTANDER CONSUMER USA HOLDINGS INC.,GA,300XX,Servicemember,NaN,Web,10/13/2019,Closed with explanation,Yes,NaN,3404213
3,08/19/2015,Mortgage,Conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account",NaN,NaN,Company chooses not to provide a public response,WELLS FARGO & COMPANY,CA,94526,NaN,Consent not provided,Web,08/19/2015,Closed with explanation,Yes,No,1527601
4,03/04/2016,Credit card,NaN,Billing disputes,NaN,I am dissatisfied with the current outcome of ...,NaN,DISCOVER BANK,NV,891XX,NaN,Consent provided,Web,03/04/2016,Closed with explanation,Yes,Yes,1816726
5,03/18/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,"BANK OF AMERICA, NATIONAL ASSOCIATION",PA,175XX,NaN,NaN,Referral,03/19/2013,Closed with explanation,Yes,No,358304
6,12/21/2011,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,"BANK OF AMERICA, NATIONAL ASSOCIATION",MD,210XX,NaN,NaN,Web,12/22/2011,Closed without relief,Yes,No,7362
7,10/24/2018,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account status incorrect,NaN,Company has responded to the consumer and the ...,SYNCHRONY FINANCIAL,CA,92595,Servicemember,Consent not provided,Web,10/24/2018,Closed with explanation,Yes,NaN,3054861
8,03/03/2018,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,On XX/XX/18 an item showed up on my credit rep...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",VA,245XX,NaN,Consent provided,Web,03/03/2018,Closed with explanation,Yes,NaN,2831821
9,01/02/2019,Debt collection,I do not know,Attempts to collect debt not owed,Debt was result of identity theft,Saw on my credit report that there is a collec...,Company believes complaint caused principally ...,"Cascade Capital, LLC",FL,334XX,Older American,Consent provided,Web,01/18/2019,Closed with explanation,Yes,NaN,3114357


In [5]:
df.count()

Date received                   1420970
Product                         1420970
Sub-product                     1185804
Issue                           1420970
Sub-issue                        872404
Consumer complaint narrative     456275
Company public response          521617
Company                         1420970
State                           1397688
ZIP code                        1290110
Tags                             193935
Consumer consent provided?       817198
Submitted via                   1420970
Date sent to company            1420970
Company response to consumer    1420969
Timely response?                1420970
Consumer disputed?               768484
Complaint ID                    1420970
dtype: int64

In [6]:
df.columns

Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Consumer disputed?', 'Complaint ID'],
      dtype='object')

In [7]:
df['Company response to consumer'].unique()

array(['Closed with non-monetary relief', 'Closed with explanation',
       'Closed without relief', 'Closed with monetary relief', 'Closed',
       'In progress', 'Untimely response', 'Closed with relief', nan],
      dtype=object)

In [8]:
df['Consumer disputed?'].unique()

array(['No', nan, 'Yes'], dtype=object)

In [9]:
df['Issue'].unique()

array(['Incorrect information on credit report',
       "Problem with a credit reporting company's investigation into an existing problem",
       'Attempts to collect debt not owed',
       'Loan servicing, payments, escrow account', 'Billing disputes',
       'Loan modification,collection,foreclosure',
       'Incorrect information on your report',
       'Improper use of your report',
       'Account opening, closing, or management',
       'Problem with a purchase shown on your statement',
       'Identity theft / Fraud / Embezzlement', "Can't contact lender",
       'False statements or representation',
       'Problems when you are unable to pay',
       'Disclosure verification of debt', 'Struggling to pay your loan',
       'Deposits and withdrawals',
       "Cont'd attempts collect debt not owed",
       'Trouble during payment process', 'Unexpected or other fees',
       'Problem with a lender or other company charging your account',
       'Other transaction issues', 'Writte

In [10]:
df['Timely response?'].unique()

array(['Yes', 'No'], dtype=object)

In [11]:
df['Company public response'].unique()

array(['Company chooses not to provide a public response', nan,
       'Company has responded to the consumer and the CFPB and chooses not to provide a public response',
       'Company believes complaint caused principally by actions of third party outside the control or direction of the company',
       'Company believes it acted appropriately as authorized by contract or law',
       'Company believes complaint is the result of an isolated error',
       'Company disputes the facts presented in the complaint',
       'Company believes the complaint is the result of a misunderstanding',
       "Company can't verify or dispute the facts in the complaint",
       'Company believes complaint represents an opportunity for improvement to better serve consumers',
       'Company believes complaint relates to a discontinued policy or procedure'],
      dtype=object)

### Simple Scoring Function
After the data is profiled, a simple evaluation algorithm using 3 fields is developed.

In [12]:
# Weights for "Company Response"
response_weights = {
    'Closed with monetary relief': 3,
    'Closed with non-monetary relief': 2,
    'Closed with relief': 2,
    'Closed with explanation': 1,
    'Closed without relief': -1,
    'Closed': -1,
    'In progress': 0,
    'Untimely response': -2,
    'N/A': 0,
    'nan': 0,
    None: 0
}

# Weights for "Consumer disputed?"
disputed_weights = {
    'No': 2,
    'Yes': -2,
    'N/A': 0,
    'nan': 0,
    None: 0
}

# Weights for "Timely response?"
timely_weights = {
    'No': -2,
    'Yes': 2,
    'N/A': 0,
    'nan': 0,
    None: 0
}


def evalutate_quantitative(row)->int:
    """
    Provides score for a single record of bank consumer data looking only at the "Quantitative" fields

    :param row: a single complaint record
    :return: int score
    """
    score = 0
    score = score + timely_weights[str(row['Timely response?'])]
    score = score + disputed_weights[str(row['Consumer disputed?'])]
    score = score + response_weights[str(row['Company response to consumer'])]

    return score


### Apply Scoring Function
Each record now has a score assoviated with it. Positive for good, negative for bad.

In [13]:
begin = time.time()
df['score']=df.apply(lambda row: evalutate_quantitative(row), axis=1)
print(f"Elapsed: {round(time.time()-begin, 2)} sec")

Elapsed: 35.29 sec


In [14]:
df.head(10)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,score
0,08/09/2015,Credit reporting,NaN,Incorrect information on credit report,Information is not mine,NaN,Company chooses not to provide a public response,Experian Information Solutions Inc.,NJ,08872,NaN,Consent not provided,Web,08/09/2015,Closed with non-monetary relief,Yes,No,1509954,6
1,01/29/2019,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Was not notified of investigation status or re...,NaN,NaN,"EQUIFAX, INC.",NY,10801,NaN,Consent not provided,Web,01/30/2019,Closed with non-monetary relief,Yes,NaN,3136759,4
2,10/13/2019,Debt collection,I do not know,Attempts to collect debt not owed,Debt was result of identity theft,NaN,NaN,SANTANDER CONSUMER USA HOLDINGS INC.,GA,300XX,Servicemember,NaN,Web,10/13/2019,Closed with explanation,Yes,NaN,3404213,3
3,08/19/2015,Mortgage,Conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account",NaN,NaN,Company chooses not to provide a public response,WELLS FARGO & COMPANY,CA,94526,NaN,Consent not provided,Web,08/19/2015,Closed with explanation,Yes,No,1527601,5
4,03/04/2016,Credit card,NaN,Billing disputes,NaN,I am dissatisfied with the current outcome of ...,NaN,DISCOVER BANK,NV,891XX,NaN,Consent provided,Web,03/04/2016,Closed with explanation,Yes,Yes,1816726,1
5,03/18/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,"BANK OF AMERICA, NATIONAL ASSOCIATION",PA,175XX,NaN,NaN,Referral,03/19/2013,Closed with explanation,Yes,No,358304,5
6,12/21/2011,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,"BANK OF AMERICA, NATIONAL ASSOCIATION",MD,210XX,NaN,NaN,Web,12/22/2011,Closed without relief,Yes,No,7362,3
7,10/24/2018,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account status incorrect,NaN,Company has responded to the consumer and the ...,SYNCHRONY FINANCIAL,CA,92595,Servicemember,Consent not provided,Web,10/24/2018,Closed with explanation,Yes,NaN,3054861,3
8,03/03/2018,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,On XX/XX/18 an item showed up on my credit rep...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",VA,245XX,NaN,Consent provided,Web,03/03/2018,Closed with explanation,Yes,NaN,2831821,3
9,01/02/2019,Debt collection,I do not know,Attempts to collect debt not owed,Debt was result of identity theft,Saw on my credit report that there is a collec...,Company believes complaint caused principally ...,"Cascade Capital, LLC",FL,334XX,Older American,Consent provided,Web,01/18/2019,Closed with explanation,Yes,NaN,3114357,3


### Filter out companies with less than 30 records
We cannot be confident in ratings for companies with not enough data.

In [15]:
begin = time.time()
df_filtered = df.groupby("Company").filter(lambda row: len(row)>30)
print(f"Elapsed: {round(time.time()-begin, 2)} sec")

Elapsed: 1.21 sec


In [16]:
df_filtered.count()

Date received                   1393817
Product                         1393817
Sub-product                     1159216
Issue                           1393817
Sub-issue                        853538
Consumer complaint narrative     445067
Company public response          512627
Company                         1393817
State                           1371097
ZIP code                        1266374
Tags                             189990
Consumer consent provided?       799441
Submitted via                   1393817
Date sent to company            1393817
Company response to consumer    1393816
Timely response?                1393817
Consumer disputed?               754170
Complaint ID                    1393817
score                           1393817
dtype: int64

### Aggregate the score for each company

In [18]:
df_company = df_filtered.groupby(["Company"]).agg({"score":"mean"})

#### Bottom 10 Companies

In [19]:
df_company.sort_values("score").head(10)

,score
Company,
"Walwick, Inc",-3.437500
"Global Portfolio Recovery, LLC",-3.400000
ALTERNATIVE DISPUTE RESOLUTIONS,-3.375000
VANGUARD FUNDING LLC,-3.184211
Screening Reports Inc.,-3.125000
"Mobiloans, LLC",-2.738693
"Accelerated Receivables Management, Inc.",-2.686567
NAM National Arbitration and Mediation,-2.486842
Ameritech Financial,-2.416268


#### Top 10 Companies

In [21]:
df_company.sort_values("score", ascending=False).head(10)

,score
Company,
Express Aviation,6.518519
"Empowerment Ventures, LLC",5.935089
"Oxford Law, LLC",5.344595
"Accounts Receivable Management, Inc (NJ) (Closed)",5.269231
Turning Point Solutions LLC,5.142857
Allied Interstate LLC,5.110692
"Media Collections, Inc",5.103704
Van Ru Credit Corporation,5.059633
"The Affiliated Group, Inc",5.030769


### Aggregate the score for each product

In [22]:
df_product = df_filtered.groupby(["Product"]).agg({"score":"mean"})

In [23]:
df_product.sort_values("score")

,score
Product,
Virtual currency,2.444444
"Payday loan, title loan, or personal loan",2.861279
Vehicle loan or lease,3.062842
"Credit reporting, credit repair services, or other personal consumer reports",3.132761
"Money transfer, virtual currency, or money service",3.145225
Credit card or prepaid card,3.412896
Checking or savings account,3.421037
Debt collection,3.595150
Mortgage,3.792152
